In [11]:
import os
os.chdir("..")

In [12]:
import django
from django.conf import settings
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'mimuse.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [13]:
import json
import http.client

from songs.models import Album
from artists.models import Artist

In [14]:
conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "6e75800c20msh1e3b52b20821a7dp1194edjsn5afc9a2de8d4",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}

conn.request("GET", "/artist_albums/?id=2w9zwq3AktTeYYMuhMjju8&offset=0&limit=100", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))

{"data":{"artist":{"discography":{"albums":{"totalCount":11,"items":[{"releases":{"items":[{"id":"5aWF6mKr3bSRlMnyxA8Tip","uri":"spotify:album:5aWF6mKr3bSRlMnyxA8Tip","name":"Champagne Problems #DQH2","type":"ALBUM","date":{"year":2022,"isoString":"2022-03-11T00:00:00Z"},"coverArt":{"sources":[{"url":"https:\/\/i.scdn.co\/image\/ab67616d00001e02fec9341c25339773dfe59cb5","width":300,"height":300},{"url":"https:\/\/i.scdn.co\/image\/ab67616d00004851fec9341c25339773dfe59cb5","width":64,"height":64},{"url":"https:\/\/i.scdn.co\/image\/ab67616d0000b273fec9341c25339773dfe59cb5","width":640,"height":640}]},"playability":{"playable":true,"reason":"PLAYABLE"},"sharingInfo":{"shareId":"9N2N_z5WQlSs4-9L9NEajQ","shareUrl":"https:\/\/open.spotify.com\/album\/5aWF6mKr3bSRlMnyxA8Tip?si=9N2N_z5WQlSs4-9L9NEajQ"},"tracks":{"totalCount":8}}]}},{"releases":{"items":[{"id":"62XwEYC2iJIX37oZlJbsvi","uri":"spotify:album:62XwEYC2iJIX37oZlJbsvi","name":"Champagne Problems #DQH1","type":"ALBUM","date":{"year":2

In [15]:
import html
import bs4

In [16]:
conn = http.client.HTTPSConnection("spotify23.p.rapidapi.com")

headers = {
    'X-RapidAPI-Key': "6e75800c20msh1e3b52b20821a7dp1194edjsn5afc9a2de8d4",
    'X-RapidAPI-Host': "spotify23.p.rapidapi.com"
}


data = Artist.objects.all()

for i in data:
    artist_id = str(i)
    print(i)
    conn.request("GET", "/artist_albums/?id={artist_id}".format(artist_id=artist_id), headers=headers)
    res = conn.getresponse()
    datas = json.loads(res.read().decode('utf-8'))
    print(datas)
    albums_list = []
    num_albums = datas['data']['artist']['discography']['albums']['totalCount']
    x = 0
    for album in datas['data']['artist']['discography']['albums']['items']:
        this = Album(api_id = album['releases']['items'][0]['id'])
        this.save()
        artist = Artist.objects.get(api_id=artist_id)
        artist.albums.add(this)
        artist.save()
        print(albums_list)

In [17]:
bulk = Album.objects.bulk_create(albums_list)

NameError: name 'albums_list' is not defined